In [1]:
import pandas as pd
import numpy as np


# Full datafiles explanation can be found in:
# https://web.archive.org/web/20180506035658/http://lisp.vse.cz/pkdd99/Challenge/berka.htm

In [2]:
# Read Files
account_df = pd.read_csv("account.txt", sep = ";", header = 0, names = ["account_id", "branch_dist_id", "frequency", "acc_create_date"])
card_df = pd.read_csv("card.txt", sep = ";", header = 0, names = ["card_id", "disp_id", "card_type", "issued"])
client_df = pd.read_csv("client.txt", sep = ";", header = 0, names = ["client_id", "birth_number", "acc_district_id"])
disp_df = pd.read_csv("disp.txt", sep = ";", header = 0, names = ["disp_id", "client_id", "account_id", "ownership_type"])
district_df = pd.read_csv("district.txt", sep = ";")
loan_df = pd.read_csv("loan.txt", sep = ";", header = 0, names = ["loan_id","account_id","loan_date","loan_amt","duration","payments","status"])
order_df = pd.read_csv("order.txt", sep = ";")
trans_df = pd.read_csv("trans.txt", sep = ";", dtype={"k_symbol" : object, "bank": object})

In [3]:
'''
Clean empty values
'''

account_df = account_df.replace(" ", np.nan)
card_df = card_df.replace(" ", np.nan)
client_df = client_df.replace(" ", np.nan)
disp_df = disp_df.replace(" ", np.nan)
district_df = district_df.replace(" ", np.nan)
loan_df = loan_df.replace(" ", np.nan)
order_df = order_df.replace(" ", np.nan)
trans_df = trans_df.replace(" ", np.nan)

In [4]:
# Formatting and Extracting gender

client_df["day"] = client_df["birth_number"].astype(str).str.slice(start=4)
client_df["month"] = client_df["birth_number"].astype(str).str.slice(start=2, stop=4).astype(int)
client_df["year"] = "19" + client_df["birth_number"].astype(str).str.slice(stop=2)
client_df["gender_male"] = np.where(client_df["month"] > 12, 0, 1)
client_df["month"] = np.where(client_df["month"] > 12, client_df["month"] - 50, client_df["month"])
client_df["birth_number"] = client_df["year"].astype(str) + client_df["month"].map("{:02}".format).astype(str) + client_df["day"].astype(str)
client_df["birth_number"] = client_df["birth_number"].astype(int)
client_df = client_df.drop(["day", "month", "year"], axis = 1)

In [5]:
'''
Convert into date columns
'''

#trans_df["date"] = pd.to_datetime(trans_df["date"], format="%y%m%d")
account_df["acc_create_date"] = pd.to_datetime(account_df["acc_create_date"], format="%y%m%d")
client_df["birth_number"] = pd.to_datetime(client_df["birth_number"], format="%Y%m%d")
loan_df["loan_date"] = pd.to_datetime(loan_df["loan_date"], format="%y%m%d")
#loan_df["loan_expiry_date"] = loan_df["loan_date"] + pd.TimedeltaIndex(loan_df['duration'], unit='M')
loan_df["loan_expiry_date"] = loan_df["loan_date"] + loan_df['duration'].values.astype("timedelta64[M]") 
loan_df["loan_expiry_date"] = loan_df["loan_expiry_date"].dt.date
#+ df['offset'].astype('timedelta64[D]'))
trans_df["date"] = pd.to_datetime(trans_df["date"], format="%y%m%d")

In [6]:
#loan_df.head(10)

In [7]:
'''
Remove accounts that did not take loans
'''

loan_accs = loan_df["account_id"].tolist()
trans_df = trans_df[trans_df["account_id"].isin(loan_accs)]

In [8]:
# Loan Returned
loan_a = loan_df.loc[(loan_df["status"] == "A") | (loan_df["status"] == "C")]

# Loan Defaulted
loan_b = loan_df.loc[(loan_df["status"] == "B") | (loan_df["status"] == "D")]

In [9]:
labels_df = loan_df.copy()
labels_df["target"] = np.where( (labels_df["status"] == "A") | (labels_df["status"] == "C"), 0, 1)
labels_df = labels_df.sort_values("account_id")
labels_df = labels_df.drop(["loan_id", "loan_date", "loan_amt", "duration", "payments", "status", "loan_expiry_date"], axis = 1)

# Save file
labels_df.to_csv("targets.csv", index = False)

In [10]:
# Join
trans_df = trans_df.merge(account_df, on="account_id", how = "left")
trans_df = trans_df.drop(["frequency"], axis = 1)

In [11]:
# Join
trans_df = trans_df.merge(disp_df, on="account_id", how = "left")
trans_df = trans_df[trans_df["ownership_type"] == "OWNER"]
trans_df = trans_df.drop(["ownership_type"], axis = 1)
trans_df = trans_df.merge(client_df, on="client_id", how = "left")
#trans_df = trans_df.merge(card_df, on="disp_id", how = "left")
trans_df["age"] = trans_df["date"].dt.year - trans_df["birth_number"].dt.year

In [12]:
#trans_df.head()

In [13]:
trans_df = trans_df.merge(loan_df, on="account_id", how = "left")
trans_df["existing_loan"] = np.where( (trans_df["loan_date"] <= trans_df["date"]) & (trans_df["date"] < trans_df["loan_expiry_date"]), 1, 0)
trans_df["loan_amount"] = np.where((trans_df["loan_date"] <= trans_df["date"]) & (trans_df["date"] < trans_df["loan_expiry_date"]), trans_df["loan_amt"], 0)

In [14]:
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     display(trans_df[trans_df["account_id"] == 5270].head())

In [15]:
# Groupby account
# sort date, trans_id
trans_df = trans_df.sort_values(['account_id','date',"trans_id"])
#trans_df = trans_df.set_index("date")
#trans_df["days_since_acc_create"] = 0
#trans_df["days_since_previous_trans"] = 0
#trans_df["unique_counter_parties"] =  0
trans_df["days_since_prev_trans"] = trans_df.groupby("account_id")["date"].diff().dt.days
trans_df["days_since_prev_trans"] = trans_df["days_since_prev_trans"].fillna(0)
trans_df["days_since_prev_trans"] = trans_df["days_since_prev_trans"].astype(int)
#grp_acc_crt = trans_df.groupby("account_id")["date"].apply((df["Date"] - df["Date"].min()).dt.days)
#df.reset_index(inplace=True)

In [16]:
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     display(trans_df.head())

In [17]:
grp_acc_crt = trans_df.groupby("account_id").apply(lambda x: (x["date"] - x["acc_create_date"]).dt.days).reset_index()
grp_acc_crt = grp_acc_crt.set_index("level_1")
grp_acc_crt = grp_acc_crt.drop("account_id", axis = 1)
grp_acc_crt.index.name = None
grp_acc_crt.columns = ["days_since_acc_creation"]
trans_df = trans_df.join(grp_acc_crt, how="left")

In [18]:
#trans_df["account"] = trans_df["account"].replace(0.0, np.NaN)
trans_df["account"] = trans_df["account"].fillna(0.0)
# m = trans_df["account"].notna()
# trans_df.loc[m, 'unique_acc_count']= (trans_df[m].groupby("account_id")["account"].transform(lambda x: pd.factorize(x)[0]))
# trans_df["unique_acc_count"] = trans_df["unique_acc_count"] + 1

In [19]:
trans_df['unique_acc_count'] = (trans_df.drop_duplicates(['account_id',"account"])
                                .groupby('account_id')
                                .cumcount())
trans_df['unique_acc_count'] = trans_df['unique_acc_count'].ffill().astype(int)
trans_df["unique_acc_count"] = trans_df["unique_acc_count"] + 1

In [20]:
trans_df["num_trans_same_party"] = trans_df.groupby(["account_id","account"]).cumcount()+1

In [21]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(trans_df.head(30))

,trans_id,account_id,date,type,operation,amount,balance,k_symbol,bank,account,branch_dist_id,acc_create_date,disp_id,client_id,birth_number,acc_district_id,gender_male,age,loan_id,loan_date,loan_amt,duration,payments,status,loan_expiry_date,existing_loan,loan_amount,days_since_prev_trans,days_since_acc_creation,unique_acc_count,num_trans_same_party
44,276,2,1993-02-26,PRIJEM,VKLAD,1100.0,1100.0,NaN,NaN,0.0,1,1993-02-26,2,2,1945-02-04,1,1,48,4959,1994-01-05,80952,24,3373.0,A,1996-01-05,0,0,0,0,1,1
84,279,2,1993-03-12,PRIJEM,PREVOD Z UCTU,20236.0,21336.0,NaN,ST,66487163.0,1,1993-02-26,2,2,1945-02-04,1,1,48,4959,1994-01-05,80952,24,3373.0,A,1996-01-05,0,0,14,14,2,1
129,697,2,1993-03-28,PRIJEM,VKLAD,3700.0,25036.0,NaN,NaN,0.0,1,1993-02-26,2,2,1945-02-04,1,1,48,4959,1994-01-05,80952,24,3373.0,A,1996-01-05,0,0,16,30,2,2
168,3530483,2,1993-03-31,PRIJEM,NaN,13.5,25049.5,UROK,NaN,0.0,1,1993-02-26,2,2,1945-02-04,1,1,48,4959,1994-01-05,80952,24,3373.0,A,1996-01-05,0,0,3,33,2,3
204,280,2,1993-04-12,PRIJEM,PREVOD Z UCTU,20236.0,45285.5,NaN,ST,66487163.0,1,1993-02-26,2,2,1945-02-04,1,1,48,4959,1994-01-05,80952,24,3373.0,A,1996-01-05,0,0,12,45,2,2
261,698,2,1993-04-27,VYDAJ,VYBER,11000.0,34285.5,NaN,NaN,0.0,1,1993-02-26,2,2,1945-02-04,1,1,48,4959,1994-01-05,80952,24,3373.0,A,1996-01-05,0,0,15,60,2,4
326,3530484,2,1993-04-30,PRIJEM,NaN,109.5,34394.9,UROK,NaN,0.0,1,1993-02-26,2,2,1945-02-04,1,1,48,4959,1994-01-05,80952,24,3373.0,A,1996-01-05,0,0,3,63,2,5
389,281,2,1993-05-12,PRIJEM,PREVOD Z UCTU,20236.0,54630.9,NaN,ST,66487163.0,1,1993-02-26,2,2,1945-02-04,1,1,48,4959,1994-01-05,80952,24,3373.0,A,1996-01-05,0,0,12,75,2,3
465,699,2,1993-05-27,VYDAJ,VYBER,17600.0,37030.9,NaN,NaN,0.0,1,1993-02-26,2,2,1945-02-04,1,1,48,4959,1994-01-05,80952,24,3373.0,A,1996-01-05,0,0,15,90,2,6
552,3530485,2,1993-05-31,PRIJEM,NaN,144.7,37175.6,UROK,NaN,0.0,1,1993-02-26,2,2,1945-02-04,1,1,48,4959,1994-01-05,80952,24,3373.0,A,1996-01-05,0,0,4,94,2,7


In [22]:
#trans_df["target"] = np.where(trans_df["status"] == "A", )
cols_del = ["trans_id", "date", "account", "acc_create_date", "disp_id", "client_id", "birth_number",
           "loan_id", "loan_date", "loan_amt", "duration", "payments", "loan_expiry_date", "status"]
trans_df = trans_df.drop(cols_del, axis = 1)

In [23]:
trans_df.to_csv("data.csv", index = False)

In [24]:
#trans_df.head(30)

In [25]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(trans_df.tail(10))

,account_id,type,operation,amount,balance,k_symbol,bank,branch_dist_id,acc_district_id,gender_male,age,existing_loan,loan_amount,days_since_prev_trans,days_since_acc_creation,unique_acc_count,num_trans_same_party
185647,11362,VYDAJ,VYBER,14.6,27722.8,SLUZBY,NaN,67,67,0,36,1,129408,3,1143,6,180
186982,11362,PRIJEM,NaN,128.7,27737.4,UROK,NaN,67,67,0,36,1,129408,0,1143,6,181
187343,11362,PRIJEM,VKLAD,24212.0,51934.8,NaN,NaN,67,67,0,36,1,129408,5,1148,6,182
187574,11362,VYDAJ,PREVOD NA UCET,129.0,51805.8,NaN,KL,67,67,0,36,1,129408,1,1149,6,34
187762,11362,VYDAJ,PREVOD NA UCET,330.0,51475.8,POJISTNE,ST,67,67,0,36,1,129408,1,1150,6,34
188002,11362,VYDAJ,PREVOD NA UCET,56.0,51419.8,NaN,MN,67,67,0,36,1,129408,1,1151,6,34
188406,11362,VYDAJ,PREVOD NA UCET,4780.0,46639.8,SIPO,YZ,67,67,0,36,1,129408,2,1153,6,34
188792,11362,VYDAJ,PREVOD NA UCET,5392.0,41247.8,UVER,MN,67,67,0,36,1,129408,2,1155,6,24
190099,11362,VYDAJ,VYBER,2880.0,38367.8,NaN,NaN,67,67,0,36,1,129408,7,1162,6,183
191503,11362,PRIJEM,NaN,162.8,38530.6,UROK,NaN,67,67,0,36,0,0,12,1174,6,184
